In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import cv2
import numpy as np
import tensorflow as tf
from NumberPlate_OCR import ocr
from yolov3.utils import detect_image, detect_realtime, detect_video, Load_Yolo_model, detect_video_realtime_mp
from yolov3.configs import *
from yolov3.yolov4 import Create_Yolo
from PIL import Image
import pytesseract
import argparse
import cv2
import os
import numpy as np

In [2]:
yolo = Create_Yolo(input_size=YOLO_INPUT_SIZE, CLASSES=TRAIN_CLASSES)

In [3]:
yolo.load_weights("./checkpoints/yolov4_custom")

In [22]:
image_path   = "./IMAGES/9.jpg"

In [23]:
bb = detect_image(yolo, image_path, "./IMAGES/plate_1_detect.jpg", 
             input_size=YOLO_INPUT_SIZE, show=True, CLASSES=TRAIN_CLASSES, rectangle_colors=(255,0,0))

error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-52oirelq\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [6]:
bb


[array([  0.79700911,  10.78493118, 282.24310303, 166.1799469 ,
          0.88488376,   0.        ])]

In [7]:
img = cv2.imread(image_path)
number_plate_img = img[int(bb[1]):int(bb[3]),int(bb[0]):int(bb[2])]
img = cv2.rectangle(img, (int(bb[0]),int(bb[1])), 
                                    (int(bb[0]+slot[2]),int(bb[1]+bb[3])), (255, 0, 0), )


IndexError: list index out of range

In [33]:
cv2.imshow("qwe",img)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite("cropped.png",number_plate_img)

In [41]:
ocr("cropped.png")

'MH20DV2366'

In [47]:

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

path = "cropped.png"

img_name = "KA67PN9942.png"

filename = "cropped.png"

gray = cv2.imread(filename, 0)
gray = cv2.resize( gray, None, fx = 3, fy = 3, interpolation = cv2.INTER_CUBIC)
ocr("cropped.png")

blur = cv2.GaussianBlur(gray, (5,5), 0)
gray = cv2.medianBlur(gray, 3)
# perform otsu thresh (using binary inverse since opencv contours work better with white text)
ret, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)
# cv2.imshow("Otsu", thresh)
cv2.waitKey(0)
rect_kern = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))

 
dilation = cv2.dilate(thresh, rect_kern, iterations = 1)

try:
    contours, hierarchy = cv2.findContours(dilation, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
except:
    ret_img, contours, hierarchy = cv2.findContours(dilation, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
sorted_contours = sorted(contours, key=lambda ctr: cv2.boundingRect(ctr)[0])

# create copy of image
im2 = gray.copy()

plate_num = ""
# loop through contours and find letters in license plate
for cnt in sorted_contours:
    x,y,w,h = cv2.boundingRect(cnt)
    height, width = im2.shape
    
    # if height of box is not a quarter of total height then skip
    if height / float(h) > 6: continue
    ratio = h / float(w)
    # if height to width ratio is less than 1.5 skip
    if ratio < 1.5: continue
    area = h * w
    # if width is not more than 25 pixels skip
    if width / float(w) > 15: continue
    # if area is less than 100 pixels skip
    if area < 100: continue
    # draw the rectangle
    rect = cv2.rectangle(im2, (x,y), (x+w, y+h), (0,255,0),2)
    roi = thresh[y-5:y+h+5, x-5:x+w+5]
    roi = cv2.bitwise_not(roi)
    roi = cv2.medianBlur(roi, 5)
    #cv2.imshow("ROI", roi)
    #cv2.waitKey(0)
    text = pytesseract.image_to_string(roi, config='-c tessedit_char_whitelist=0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ --psm 8 --oem 3')
    #print(text)
    plate_num += text
print(ocr("cropped.png"))
# cv2.imshow("Character's Segmented", im2)
cv2.waitKey(0)
cv2.destroyAllWindows()

MH20DV2366
